In [8]:
import re
from posextract import grammatical_triples

In [113]:
def count_words(clean_data):
    ngrams_dict = {}
    for ngram in clean_data:
        if ngram in ngrams_dict:
            ngrams_dict[ngram] += 1
        else:
            ngrams_dict[ngram] = 1
    return ngrams_dict


def import_and_clean_data(path, news_source):
    fname = path + news_source + '_full_article_text_12-27-2021.txt'
    replace_dict = {'\n':'', '   ':'', 'Loading...':'', 'U.S.':'United States'}
  
    file = open(fname)
    lines = [line for line in file.readlines() if line.strip()] # do i want this if statement
    file.close()
  
    out = ' '.join(lines)
  
    for k, v in replace_dict.items():
      out = out.replace(k, v)
    
    return re.split('(?<=\.)(?=[A-Z])', out)
  

def extract_triples(path, news_sources, triples_dict, keyword_dict):
  
  for news_source in news_sources:

    clean_data = import_and_clean_data(path, news_source)

    triples = grammatical_triples.extract(clean_data)

    for elem in keyword_dict[news_source]:
      for triple in triples:
         if re.match(elem, str(triple), re.IGNORECASE):
          triples_dict[news_source].append(str(triple).title())

  return triples_dict

In [114]:
path = '/home/stephbuon/projects/entascope/data/12-27-2021/'
news_sources = ['FOX', 'NPR']
triples_dict = {news_sources[0]: [], news_sources[1]: []}
#people_dict = {'FOX': ['Trump', 'Biden'], 'NPR': ['Trump', 'Biden']}
people_dict = {'FOX': ['Biden', 'Trump', 'Fauci', 'Psaki', 'Blasio', 'Cuomo', 'Desantis', 'Newsom', 'Powell', 'Hochul','Mayorkas', 'Brown', 'Harris', 'Walensky', 'Harris', 'Siegel', 'Horak'], 
               'NPR': ['Biden', 'Collins', 'Fauci', 'Osterholm', 'Hochul', 'Chu', 'Kilgore', 'Walensky', 'Hochul', 'Tedros', 'Kerkhove', 'Blasio', 'Newsom', 'Austin', 'Kirby', 'Brown', 'Trump', 'Allie', 'Ghebreyesus', 'Dolsten', 'Mammen', 'Pollack', 'Ryan']}

extracted_triples = extract_triples(path, news_sources, triples_dict, people_dict)

In [116]:
subject_verb_pairs = []

for string in extracted_triples['FOX']:
    subject_verb_pairs.append(' '.join(string.split()[0:2]))

triples_count = count_words(subject_verb_pairs)

In [ ]:
# blame, tout, accuse 

In [117]:
for k,v in triples_count.items():
    if v > 1:
        print(k,v)

Biden Took 15
Biden Announced 26
Biden Expanded 4
Bidenand Were 2
Bidenis Address 3
Bidenis Suggested 2
Biden Hit 2
Biden Speaks 37
Biden Had 2
Biden Resisted 2
Biden Need 2
Biden Tested 5
Bidenhimself Pushed 2
Biden Use 2
Biden Delivers 10
Biden Creating 2
Biden Cause 4
Biden Attacked 2
Biden Blames 4
Biden Promised 4
Biden Acknowledged 3
Biden Listens 3
Biden Walks 5
Biden Spreading 2
Biden Pushed 3
Biden Make 2
Biden Issued 4
Biden Followed 3
Biden Announces 8
Biden Not 18
Biden Flunked 2
Biden Face 2
Biden Rescind 5
Biden Planning 2
Biden Using 2
Biden Finding 2
Biden Needs 3
Biden N'T 4
Bidenhas Defended 2
Biden Told 4
Biden Telling 3
Biden Put 3
Biden Were 2
Biden Seeing 2
Biden Says 2
Biden Lost 5
Biden Rallied 2
Biden Declares 4
Biden Crave 3
Biden Shared 2
Biden Made 3
Biden Lifting 2
Biden Answers 3
Biden Limit 2
Biden Approved 6
Biden Been 3
Biden Attend 6
Biden Talks 4
Biden Take 4
Biden Imposing 2
Biden Requiring 4
Biden Used 4
Biden Become 2
Biden Come 3
Biden Mandate 4
B